In [ ]:
%env KITTI_ROOT=/data/kitti_lidar
!export KITTI_ROOT=/data/kitti_lidar

In [ ]:
%env NUMBA_WARNINGS=0

### To use different copy of `second.pytorch`

In [ ]:
# %%bash
# cd /home/pytorch/external/ && git clone https://github.com/jitrc/second.pytorch -b quick_fix
# export SECOND_API=/home/pytorch/external/second.pytorch/second
# export PYTHONPATH=/home/pytorch/external/second.pytorch:${PYTHONPATH}

In [ ]:
# %env SECOND_API=/home/pytorch/external/second.pytorch/second
# %env PYTHONPATH=/home/pytorch/external/second.pytorch:${PYTHONPATH}

In [ ]:
!ls ${SECOND_API}

In [ ]:
!ls ${KITTI_ROOT}

In [ ]:
!tree --filelimit 10 ${KITTI_ROOT}

# Prepare dataset

In [ ]:
!mkdir -p ${KITTI_ROOT}/training/velodyne_reduced/
!mkdir -p ${KITTI_ROOT}/testing/velodyne_reduced/

In [ ]:
!python ${SECOND_API}/create_data.py kitti_data_prep ${KITTI_ROOT}

In [ ]:
!ls -l --block-size=M /data/kitti_lidar/*.pkl

# Configure

In [ ]:
!ls -R ${SECOND_API}/configs

In [ ]:
%env KITTI_ROOT=/data/kitti_lidar
%env MODEL_OUT_PATH=/data/kitti_lidar/trained_model
%env MODEL_NAME=car.lite
%env MODEL_NAME_FLAT=car_lite

In [ ]:
%%bash
# Use this block to copy/paste in terminal 

# export KITTI_ROOT='/data/kitti_lidar'
# export MODEL_OUT_PATH="${KITTI_ROOT}/trained_model"
# export MODEL_NAME='car.fhd' # 'all.fhd', 'car.fhd', `pointpillars/cars/xyres_16` ...
# export MODEL_NAME_FLAT=$(echo ${MODEL_NAME//\//_}| tr "." _)

In [ ]:
!echo $MODEL_NAME_FLAT

## Update config file

In [ ]:
%%bash
mkdir -p ${MODEL_OUT_PATH}
cp ${SECOND_API}/configs/${MODEL_NAME}.config ${MODEL_OUT_PATH}/train.config

sed -i "s|kitti_root_path:.*|kitti_root_path: \"${KITTI_ROOT}\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|kitti_info_path:.*/kitti_infos_train.pkl\"|kitti_info_path: \"${KITTI_ROOT}/kitti_infos_train.pkl\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|kitti_info_path:.*/kitti_infos_val.pkl\"|kitti_info_path: \"${KITTI_ROOT}/kitti_infos_val.pkl\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|kitti_info_path:.*/kitti_infos_test.pkl\"|kitti_info_path: \"${KITTI_ROOT}/kitti_infos_test.pkl\"|g" ${MODEL_OUT_PATH}/train.config
sed -i "s|database_info_path:.*|database_info_path: \"${KITTI_ROOT}/kitti_dbinfos_train.pkl\"|g" ${MODEL_OUT_PATH}/train.config

# sed -i "s|batch_size:.*|batch_size: 6|g" ${MODEL_OUT_PATH}/train.config
# sed -i "s|num_workers:.*|num_workers: 4|g" ${MODEL_OUT_PATH}/train.config

cat ${MODEL_OUT_PATH}/train.config

# Evalute Pretrained

In [ ]:
!ls /data/pretrained_models/Pointpillars_models_v1.5

In [ ]:
!python ${SECOND_API}/pytorch/train.py evaluate --config_path=${MODEL_OUT_PATH}/train.config --model_dir=/data/pretrained_models/Pointpillars_models_v1.5/${MODEL_NAME_FLAT}

# Training

## Tensorboard

In [ ]:
%load_ext tensorboard.notebook
%tensorboard --logdir /data/kitti_lidar/trained_model

##  Prepate output directory

In [ ]:
!echo ${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}
!rm -rf ${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}

## Train with single GPU

In [ ]:
!python ${SECOND_API}/pytorch/train.py train --config_path=${MODEL_OUT_PATH}/train.config --model_dir=${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}

## Train with Multi GPU
not tested

In [ ]:
# !CUDA_VISIBLE_DEVICES=0,1 python ${SECOND_API}/pytorch/train.py train --config_path=${MODEL_OUT_PATH}/train.config --model_dir=${MODEL_OUT_PATH}/${MODEL_NAME_FLAT} --multi_gpu=True

# Evaluate

In [ ]:
!python ${SECOND_API}/pytorch/train.py evaluate --config_path=${MODEL_OUT_PATH}/train.config --model_dir=${MODEL_OUT_PATH}/${MODEL_NAME_FLAT}